In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import numpy as np
import pandas as pd
import torch as torch

In [36]:
x_f = torch.rand(500, 450, 10)

In [37]:
x_f.shape

torch.Size([500, 450, 10])

In [46]:
idx = torch.add(torch.arange(400).unsqueeze(axis = 1), torch.arange(50).unsqueeze(axis = 0))

In [47]:
idx

tensor([[  0,   1,   2,  ...,  47,  48,  49],
        [  1,   2,   3,  ...,  48,  49,  50],
        [  2,   3,   4,  ...,  49,  50,  51],
        ...,
        [397, 398, 399,  ..., 444, 445, 446],
        [398, 399, 400,  ..., 445, 446, 447],
        [399, 400, 401,  ..., 446, 447, 448]])

In [58]:
expanded = x_f[:, idx, :] # expanded 

In [86]:
expanded.shape

torch.Size([500, 400, 50, 10])

In [59]:
expanded.view(500, 400, 500).shape # expand

torch.Size([500, 400, 500])

In [87]:
hf1 = nn.Linear(500, 50)(x_f[:, idx, :].view(-1, 400, 500))

In [88]:
hf1.shape

torch.Size([500, 400, 50])

In [103]:
x_t = torch.rand(500, 400, 30)

In [104]:
c1 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 1)
c2 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 2)
c3 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 4)
c4 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 8)
c5 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 16)
c6 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 32)

In [105]:
x_t = c1(x_t.permute(0,2,1))
x_t = c6(c5(c4(c3(c2(x_t)))))

In [106]:
x_t.permute(0, 2, 1).shape

torch.Size([500, 349, 30])

In [107]:
x = torch.rand(500, 450, 50)

In [5]:
x.shape

torch.Size([500, 450, 20])

In [108]:
x_s = x[:, :, 0:10]
x_t = x[:, :, 10:40]
x_f = x[:, :, 40:]

In [109]:
x_s.shape, x_t.shape, x_f.shape

(torch.Size([500, 450, 10]),
 torch.Size([500, 450, 30]),
 torch.Size([500, 450, 10]))

In [110]:
s = nn.Linear(10, 50)(x_s)



In [115]:
c1 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 1)
c2 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 2)
c3 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 4)
c4 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 8)
c5 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 16)
c6 = nn.Conv1d(30, 30, 2, padding = 1, dilation = 32)

In [112]:
t = c6(c5(c4(c3(c2(c1(x_t.permute(0, 2, 1))))))).permute(0, 2, 1)

In [113]:
s.shape

torch.Size([500, 450, 50])

In [116]:
t.shape

torch.Size([500, 399, 50])

In [94]:
encoding = torch.cat((s, t), axis = 1)

In [95]:
encoding.shape

torch.Size([500, 849, 50])

In [96]:
ht = torch.cat((hf1, encoding), axis = 1)

In [97]:
ht.shape

torch.Size([500, 1249, 50])

In [99]:
nn.Linear(50, 1000)(ht).shape

torch.Size([500, 1249, 1000])

In [85]:
nn.Linear(30, 100)(ht).unsqueeze(axis = 2).repeat(50, 2)

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor

In [102]:
500*400*50*20

200000000